# Piping Chains and the RunnablePassthrough Class

In [14]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [15]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [16]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [17]:
chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [18]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  seed=365,
                  temperature = 0,
                  max_tokens = 100)

In [19]:
string_parser = StrOutputParser()

In [20]:
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

In [21]:
print(chain_tools.invoke({'job title':'data scientist'}))
print(chain_strategy.invoke({'tools':'''
1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop
'''}))

{'tools': '1. Python\n2. R Programming\n3. SQL\n4. Tableau\n5. Hadoop'}
1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and error handling. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like HackerRank or LeetCode. Work on small projects to apply what you've learned.

2. R Programming: Begin with the basics of


In [22]:
chain_combined = chain_tools | chain_strategy

In [23]:
print(chain_combined.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, operators, loops, and functions. Once you have a good understanding of these, move on to more advanced topics like classes and objects, exception handling, and file I/O operations. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like HackerRank or LeetCode. Work on small projects to apply what you've learned.

2. R Programming


In [24]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} |
              chat_template_strategy | chat | string_parser)

In [25]:
print(chain_long.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, operators, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode, HackerRank, or Codewars. Work on small projects, such as a simple calculator or a to-do list application, to apply what you've learned. Gradually move on to more complex topics like object-oriented programming, data structures


In [26]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       